In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
from datetime import timedelta
import matplotlib.dates as mdates
import sumolib
import osmnx as ox

import sys
import os
import numpy as np

from fastsim import simdrive, vehicle, cycle
from fastsim import parameters as params

from mappymatch.constructs.geofence import Geofence
from mappymatch.utils.plot import plot_geofence
from mappymatch.maps.nx.nx_map import NxMap, NetworkType

import traci
import time
import csv

import xml.etree.ElementTree as ET
import xml.dom.minidom

2024-02-05 16:30:38.964 | calibration.py:38 | WARNING: No module named 'pymoo'
Try running `pip install pymoo==0.6.0.1` to use all features in `fastsim.calibration`


In [2]:
print(os.environ['PATH'])

/common/software/install/migrated/anaconda/miniconda3_4.8.3-jupyter/bin:/sbin:/bin:/usr/sbin:/usr/bin:/opt/puppetlabs/bin:/opt/msi/sbin:/opt/msi/bin:/usr/local/bin:/usr/local/sbin


In [3]:
os.environ['PATH'] += ":/home/shekhars/yang7492/.conda/envs/syntheticData/lib/python3.8/site-packages/sumo/bin"

In [4]:
# Set SUMO_HOME
os.environ['SUMO_HOME'] = '/home/shekhars/yang7492/.conda/envs/syntheticData/lib/python3.8/site-packages/sumo'

In [5]:
data_folder = "data/synthetic/Murphy"

In [6]:
csv_name = "TL5-218_2020W33_processed.csv"
file_name = os.path.join(data_folder, csv_name)

In [7]:
def read_data(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Convert string representations of lists or tuples back to lists or tuples
    # Use ast.literal_eval safely evaluate a string containing a Python literal or container display
    df['altitude_profile'] = df['altitude_profile'].apply(ast.literal_eval)
    df['velocity_profile'] = df['velocity_profile'].apply(ast.literal_eval)
    df['weight'] = df['weight'].apply(ast.literal_eval)
    df['total_fuel'] = df['total_fuel'].apply(ast.literal_eval)
    df['trajectory'] = df['trajectory'].apply(ast.literal_eval)
    df['matched_path'] = df['matched_path'].apply(ast.literal_eval)
    df['coordinate_id'] = df['coordinate_id'].apply(ast.literal_eval)
    df['road_id'] = df['road_id'].apply(ast.literal_eval)
    
    # Remove fractional seconds from 'time'
    df['trip_start_time'] = df['trip_start_time'].str.split('.').str[0]
    # Convert 'time' to datetime while ignoring fractional seconds
    df['trip_start_time'] = pd.to_datetime(df['trip_start_time'], format='%Y-%m-%d %H:%M:%S')

    df['trip_end_time'] = df['trip_end_time'].str.split('.').str[0]
    df['trip_end_time'] = pd.to_datetime(df['trip_end_time'], format='%Y-%m-%d %H:%M:%S')
    
    df['fastsim_velocity'] = df['fastsim_velocity'].apply(lambda x: [float(i) for i in x.split()])
    df['fastsim_power'] = df['fastsim_power'].apply(lambda x: [float(i) for i in x.split()])
    df['sumo_velocity'] = df['sumo_velocity'].apply(lambda x: [float(i) for i in x.split()])
    df['sumo_path'] = df['sumo_path'].str.split()
    return df

In [8]:
processed_df = read_data(file_name)

In [9]:
processed_df.head(2)

,trip_start_time,trip_end_time,travel_time,altitude_profile,velocity_profile,weight,total_fuel,ambTemperature,trajectory,matched_path,coordinate_id,road_id,fastsim_velocity,fastsim_power,sumo_path,sumo_velocity
0,2020-08-10 12:28:47,2020-08-10 13:13:17,2670.0,"[255.7, 255.2, 254.7, 254.2, 253.8, 252.9, 252...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[30000.0, 30000.0, 30000.0, 30000.0, 30000.0, ...","[4.15, 4.15, 4.05, 4.1, 3.75, 3.8, 5.3, 6.25, ...",15.188,"[(44.788085, -93.462496), (44.78809, -93.46249...","[(8146751356, 188063558, 0), (188063558, 18805...","[202, 203, 204, 205, 206, 207, 208, 209, 210, ...","[(188055018, 9421214164, 0), (188055018, 94212...","[0.0, 5.686715111455639, 11.393352186998952, 1...","[0.0, 95.82343083537464, 229.88428843956595, 3...","[18237140#25, 18237140#25, 18237140#25, 182371...","[0.0, 5.686807374218478, 14.242079547476024, 2..."
1,2020-08-10 13:40:07,2020-08-10 13:57:35,1048.0,"[271.3, 272.8, 274.6, 276.8, 279.6, 281.9, 284...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8000.0, 8000.0, 8000.0, 8000.0, 8000.0, 8000....","[4.1, 4.05, 4.15, 4.1, 4.1, 4.25, 4.1, 4.1, 4....",16.875,"[(45.000419, -93.217756), (45.000417, -93.2177...","[(34559009, 33323569, 0), (33323569, 695998770...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(34559009, 33323569, 0), (34559009, 33323569,...","[0.0, 7.133007841291272, 12.904106082362334, 1...","[0.0, 130.41893294003683, 270.15292723619507, ...","[-202308024#14, -202308024#14, -202308024#14, ...","[0.0, 7.133123569088057, 15.077786642918364, 1..."


In [10]:
net_file = 'data/Minneapolis.net.xml'
sumo_net = sumolib.net.readNet(net_file)

In [80]:
# Download the openstreet graph within the following bounding box (the same as map-matching):
# # bounding_box = {
# #         'min_latitude': 44.403672,
# #         'max_latitude': 45.450154,
# #         'min_longitude': -94.073366,
# #         'max_longitude': -92.696696
# #     }
# geojson_file = 'results/bounding_box_mn.geojson'
# geofence = Geofence.from_geojson(geojson_file)
# osmnx_net = ox.graph_from_polygon(
#         geofence.geometry, network_type='drive'
#     )

In [85]:
# save openstreet map:
# osm_file_path = "data/maps/minneapolis.graphml"
# ox.io.save_graphml(osmnx_net, osm_file_path)

In [11]:
# load openstreet map:
osm_file_path = "data/maps/minneapolis.graphml"
osmnx_net = ox.io.load_graphml(osm_file_path)

In [12]:
node_gdf, edge_gdf = ox.utils_graph.graph_to_gdfs(osmnx_net)

In [17]:
edge_gdf.loc[(34559009,33323569,0)]

osmid                                               202308024
name                                  Northeast Hoover Street
highway                                          unclassified
oneway                                                  False
reversed                                                 True
length                                                226.354
geometry    LINESTRING (-93.2185317 45.0007914, -93.218381...
ref                                                       NaN
lanes                                                       2
access                                                    NaN
maxspeed                                               20 mph
bridge                                                    NaN
junction                                                  NaN
tunnel                                                    NaN
area                                                      NaN
width                                                     NaN
Name: (3

In [24]:
node_gdf.loc[33323569]

y                                   44.998846
x                                  -93.217768
street_count                                4
highway                                  stop
ref                                       NaN
geometry        POINT (-93.2177684 44.998846)
Name: 33323569, dtype: object

In [25]:
node_gdf.loc[34559009]

y                                    45.000791
x                                   -93.218532
street_count                                 1
highway                                    NaN
ref                                        NaN
geometry        POINT (-93.2185317 45.0007914)
Name: 34559009, dtype: object

In [26]:
34559009 in node_gdf.index

True

In [14]:
# convert the sumo_path from the edge_id representation to (origin_node_id, dest_node_id) pair

# Function to convert edge list to node list
def edge_list_to_node_list(edges, sumo_net):
    # Split the edge list string into individual edges
    node_list = []
    for edge in edges:
        if edge.startswith(':'):
            node_list.append(node_list[-1])
        else:
            edge_obj = sumo_net.getEdge(edge)  # Get the edge object
            from_node_id = edge_obj.getFromNode().getID()  # Get the ID of the origin node
            to_node_id = edge_obj.getToNode().getID()  # Get the ID of the destination node
            node_tuple = (from_node_id, to_node_id, 0)  # Create a tuple 
            node_list.append(node_tuple)
    return node_list

processed_df['sumo_node_path'] = processed_df['sumo_path'].apply(lambda x: edge_list_to_node_list(x, sumo_net))
processed_df.head(2)

,trip_start_time,trip_end_time,travel_time,altitude_profile,velocity_profile,weight,total_fuel,ambTemperature,trajectory,matched_path,coordinate_id,road_id,fastsim_velocity,fastsim_power,sumo_path,sumo_velocity,sumo_node_path
0,2020-08-10 12:28:47,2020-08-10 13:13:17,2670.0,"[255.7, 255.2, 254.7, 254.2, 253.8, 252.9, 252...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[30000.0, 30000.0, 30000.0, 30000.0, 30000.0, ...","[4.15, 4.15, 4.05, 4.1, 3.75, 3.8, 5.3, 6.25, ...",15.188,"[(44.788085, -93.462496), (44.78809, -93.46249...","[(8146751356, 188063558, 0), (188063558, 18805...","[202, 203, 204, 205, 206, 207, 208, 209, 210, ...","[(188055018, 9421214164, 0), (188055018, 94212...","[0.0, 5.686715111455639, 11.393352186998952, 1...","[0.0, 95.82343083537464, 229.88428843956595, 3...","[18237140#25, 18237140#25, 18237140#25, 182371...","[0.0, 5.686807374218478, 14.242079547476024, 2...","[(8146751356, 188063558, 0), (8146751356, 1880..."
1,2020-08-10 13:40:07,2020-08-10 13:57:35,1048.0,"[271.3, 272.8, 274.6, 276.8, 279.6, 281.9, 284...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8000.0, 8000.0, 8000.0, 8000.0, 8000.0, 8000....","[4.1, 4.05, 4.15, 4.1, 4.1, 4.25, 4.1, 4.1, 4....",16.875,"[(45.000419, -93.217756), (45.000417, -93.2177...","[(34559009, 33323569, 0), (33323569, 695998770...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(34559009, 33323569, 0), (34559009, 33323569,...","[0.0, 7.133007841291272, 12.904106082362334, 1...","[0.0, 130.41893294003683, 270.15292723619507, ...","[-202308024#14, -202308024#14, -202308024#14, ...","[0.0, 7.133123569088057, 15.077786642918364, 1...","[(34559009, 33323569, 0), (34559009, 33323569,..."


In [15]:
# extract the valid road_id, total_fuel, velocity_profile, altitude_profile using the coordinate_id
# adjust the trip_start_time accordingly

def extracted_matched_edges(row):
    starting_index = row['coordinate_id'][0]
    row["trip_start_time"] = pd.to_datetime(row['trip_start_time']) + pd.to_timedelta(starting_index, unit='s')
    row['altitude_profile'] = row['altitude_profile'][starting_index:]
    row['velocity_profile'] = row['velocity_profile'][starting_index:]
    row['weight'] = [row['weight'][0]] * len(row['weight'][starting_index:])
    row['total_fuel'] = row['total_fuel'][starting_index:]
    row['trajectory'] = row['trajectory'][starting_index:]
    row['road_id'] = row['road_id'][starting_index:]
    return row
    
processed_df = processed_df.apply(extracted_matched_edges, axis = 1)
processed_df.head(2)

,trip_start_time,trip_end_time,travel_time,altitude_profile,velocity_profile,weight,total_fuel,ambTemperature,trajectory,matched_path,coordinate_id,road_id,fastsim_velocity,fastsim_power,sumo_path,sumo_velocity,sumo_node_path
0,2020-08-10 12:32:09,2020-08-10 13:13:17,2670.0,"[238.5, 238.2, 238.0, 237.9, 237.9, 237.7, 237...","[28.691, 28.73, 28.715, 28.34, 28.141, 27.859,...","[30000.0, 30000.0, 30000.0, 30000.0, 30000.0, ...","[9.0, 8.3, 7.8, 6.15, 6.4, 6.3, 2.85, 0.55, 0....",15.188,"[(44.786858, -93.465484), (44.786858, -93.4655...","[(8146751356, 188063558, 0), (188063558, 18805...","[202, 203, 204, 205, 206, 207, 208, 209, 210, ...","[(188055018, 9421214164, 0), (188055018, 94212...","[0.0, 5.686715111455639, 11.393352186998952, 1...","[0.0, 95.82343083537464, 229.88428843956595, 3...","[18237140#25, 18237140#25, 18237140#25, 182371...","[0.0, 5.686807374218478, 14.242079547476024, 2...","[(8146751356, 188063558, 0), (8146751356, 1880..."
1,2020-08-10 13:40:07,2020-08-10 13:57:35,1048.0,"[271.3, 272.8, 274.6, 276.8, 279.6, 281.9, 284...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8000.0, 8000.0, 8000.0, 8000.0, 8000.0, 8000....","[4.1, 4.05, 4.15, 4.1, 4.1, 4.25, 4.1, 4.1, 4....",16.875,"[(45.000419, -93.217756), (45.000417, -93.2177...","[(34559009, 33323569, 0), (33323569, 695998770...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(34559009, 33323569, 0), (34559009, 33323569,...","[0.0, 7.133007841291272, 12.904106082362334, 1...","[0.0, 130.41893294003683, 270.15292723619507, ...","[-202308024#14, -202308024#14, -202308024#14, ...","[0.0, 7.133123569088057, 15.077786642918364, 1...","[(34559009, 33323569, 0), (34559009, 33323569,..."


In [16]:
def index_range_in_sumo(edge_id, row, sumo_start_index):
    new_sumo_start_index = -1
    new_sumo_end_index = -1
    if sumo_start_index < len(row["sumo_node_path"]):
        for i in range(sumo_start_index, len(row["sumo_node_path"])):
            if row["sumo_node_path"][i][0] == edge_id[0]:
                new_sumo_start_index = i
                break
        for i in range(new_sumo_start_index, len(row["sumo_node_path"])):
            if row["sumo_node_path"][i][1] == edge_id[0] and (i >= len(row["sumo_node_path"]) - 1 or row["sumo_node_path"][i][1] != row["sumo_node_path"][i+1][1]):
                new_sumo_end_index = i
                break
    return new_sumo_start_index, new_sumo_end_index+1

In [19]:
def initialize_edge_df():
    # Define the columns based on the features you want to include
    columns = [
        'edge_id', 'osmid', 'name', 'highway', 'oneway', 'length', 'geometry',
        'velocity_profile', 'liters']

    # Initialize an empty DataFrame with the defined columns
    df_edges = pd.DataFrame(columns=columns)
    
    return df_edges

In [ ]:
def convert_mph_to_kmh(speed_mph):
    if pd.isna(speed_mph):
        return speed_mph  # Return NaN as is
    if isinstance(speed_mph, str) and 'mph' in speed_mph:
        # Extract the numeric part from the string and convert it
        numeric_speed = float(speed_mph.split()[0])
        return numeric_speed * 1.60934  # Convert mph to km/h
    return speed_mph  # Return the value as is if it's already in km/h or doesn't have 'mph'

In [ ]:
def extract_endpoint_type(node_gdf, node_id):
    # revise according to the mappign
    # node_gdf.loc[node_id, 'highway']
    

In [ ]:
def compute_direction_angle(node_gdf, edge_id):
    # revise

In [20]:
def extract_osm_edge_feature(edge_gdf, edge_id):
    osm_edge_feature = edge_gdf.loc[edge_id]
    speed_limit = float(osm_edge_feature.maxspeed.strip(" ")[0]) if osm_edge_feature.maxspeed else None
    edge_features = {
        'tags': edge_id,
        'osmid': osm_edge_feature.osmid,
        'road_type': osm_edge_feature.highway,
        'speed_limit': convert_mph_to_kmh(osm_edge_feature.maxspeed)
        'length': osm_edge_feature.length,
        'lanes': osm_edge_feature.lanes,
        'bridge': 0 if pd.isna(osm_edge_feature.bridge) else 1
        'endpoint_u': extract_endpoint_type(node_gdf, edge_id[0])
        'endpoint_v': extract_endpoint_type(node_gdf, edge_id[1])
        'direction_angle': 
    }
    return edge_features

In [31]:
# for a node pair in road_id, if that both the origin node and the dest node exist in sumo_path, then it is a valid edge for training 

for i in range(len(processed_df)):
    row = processed_df.iloc[i]
    # these indics are exclusive: [start_index, end_index]
    trip_start_index = 0
    trip_end_index = 0
    sumo_start_index = 0
    sumo_end_index = 0
    
    position_of_edge = 0
    
    # output dataframe where each row represent an edge
    df_edges = initialize_edge_df()
    
    while trip_start_index < len(row["road_id"]):
              
        edge_id = row["road_id"][trip_start_index]
        # validate edge_id by checking if both the origin and the destinaiton occur in the sumo_node_path
        
        while row["road_id"][trip_end_index] == edge_id:
            trip_end_index += 1
        
        sumo_start_index, sumo_end_index = index_range_in_sumo(edge_id, row, sumo_start_index)
        
        # if the edge_id is in the sumo simulation
        if sumo_start_index >= 0:
            
            # extract simulated_features
            simulated_features = {
                'trip_id': (0, i)
                'position': position_of_edge
                'mass': 
                'elevation_change':
                'previous_orientation':
                'energy_consumption_total':
                'simulated_energy_consumption':
                'energy_consumption_per_hour':
                'energy_consumption_per_100km':
                'time':
                'speed':
                'sumo_speed':
                'fastsim_speed':
                
                'time_acc':
                'time_stage':
                'week_day':
                }
            edge_features = extract_osm_edge_feature()
            
            # Combine edge_features and additional_features
            edge_row = {**edge_features, **simulated_features}

            # Add the combined features as a new row to the DataFrame
            df_edges = df_edges.append(edge_row, ignore_index=True)
            
            # update the sumo_start_index search range
            sumo_start_index = sumo_end_index
            position_of_edge += 1
        
        # update the trip_start_index anyway
        trip_start_index = trip_end_index
        
    df_edge.head(2)
    

{<edge id="202308024#10" from="33323569" to="34559009"/>: [<sumolib.net.connection.Connection at 0x2ad68f948ee0>]}